In [ ]:
#ASSIGNMENT
#Part 1: Ingesting the data

import pandas as pd

file_path = '/content/drive/MyDrive/dataset/correct_twitter_201904.tsv'
chunk_size = 10**6
chunks = pd.read_csv(file_path, sep='\t', chunksize=chunk_size)


# Combining chunks into a single DataFrame
tweets_df = pd.concat(chunks, ignore_index=True)

# converting created_at to datetime format
# converting created_at to datetime format
tweets_df['created_at'] = pd.to_datetime(tweets_df['created_at'], errors='coerce', utc=True)

#Cleaning the data for better results
# Handling Missing Values- removing blank rows and filling blank rows as needed
tweets_df.dropna(subset=['created_at', 'text'], inplace=True)
tweets_df['like_count'].fillna(tweets_df['like_count'].mean(), inplace=True)


#Handling Duplicates
tweets_df.drop_duplicates(subset=['text'], inplace=True)

#Handling Inconsistent Data
tweets_df['text'] = tweets_df['text'].str.lower().str.strip()
tweets_df['text'] = tweets_df['text'].str.replace('teh', 'the')

#Outlier Handling (example for 'like_count')
Q1 = tweets_df['like_count'].quantile(0.25)
Q3 = tweets_df['like_count'].quantile(0.75)
IQR = Q3 - Q1
tweets_df = tweets_df[(tweets_df['like_count'] >= Q1 - 1.5 * IQR) & (tweets_df['like_count'] <= Q3 + 1.5 * IQR)]
tweets_df.head()

#print(tweets_df.head())


# PART 2: Revised Query Functionality with User Input
def query_tweets(df):
    search_term = input("Enter the term you want to search for (or 'na' for no specific term): ").strip()

    # If user inputs 'na', return a message and exit the function
    if search_term.lower() == 'na':
        print("No specific term was searched.")
        return

    # Filter tweets that contain the search term
    filtered_tweets = df[df['text'].str.contains(search_term, case=False, na=False)]

    # Checking if filtered tweets are empty
    if filtered_tweets.empty:
        print(f"No tweets found for the term '{search_term}'.")
        return

    # Ensure 'created_at' in filtered_tweets is datetime
    #filtered_tweets['created_at'] = pd.to_datetime(filtered_tweets['created_at'], errors='coerce')

#1.How many tweets were posted containing the term on each day?
    tweets_per_day = filtered_tweets['created_at'].dt.date.value_counts().sort_index()
#2.How many unique users posted a tweet containing the term?
    unique_users = filtered_tweets['author_id'].nunique()
#3.How many likes did tweets containing the term get, on average?
    average_likes = filtered_tweets['like_count'].mean()
#4.Where (in terms of place IDs) did the tweets come from?
    tweets_by_place = filtered_tweets['place_id'].value_counts().head(10)  # Show top 10 places
#5.What times of day were the tweets posted at?
    tweets_by_time = filtered_tweets['created_at'].dt.hour.value_counts().sort_index()
#6.Which user posted the most tweets containing the term?
    most_active_user = filtered_tweets['author_handle'].value_counts().idxmax()


    print(f"\nResults for search term: '{search_term}'\n")

    print("Tweets per day:")
    for date, count in tweets_per_day.items():
        print(f"{date}: {count}")
    print("\n")

    print(f"Unique users: {unique_users}")
    print(f"Average likes per tweet: {average_likes:.2f}\n")

    print("Tweets by place (place IDs - Top 10):")
    for place_id, count in tweets_by_place.items():
        print(f"{place_id}: {count}")
    print("\n")

    print("Tweets by hour:")
    for hour, count in tweets_by_time.items():
        print(f"{hour:02d}:00 - {hour:02d}:59: {count}")
    print("\n")

    print(f"Most active user: {most_active_user}\n")


query_tweets(tweets_df)

